<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Visualization" data-toc-modified-id="Visualization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Visualization</a></span></li></ul></div>

In [29]:
from konlpy import *
import numpy as np
import re
import pandas as pd
import urllib
import json
import os
import platform
from tqdm.notebook import tqdm
from lxml import etree
import string

plt.style.use("dark_background")
np.set_printoptions(precision=3)

path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Darwin":
    mpl.rc("font", family="AppleGothic")
elif platform.system() == "Windows":
    font_name = mpl.font_manager.FontProperties(fname=path).get_name()
    mpl.rc('font', family=font_name)
    
mpl.rc("axes", unicode_minus=False)

In [31]:
path = "ratings_train_okt_corpus.json"
if os.path.exists(path):
    with open(path, mode="r", encoding="utf-8") as f:
        corpus = json.load(f)
else:
    ratings_tr = pd.read_table("./Datasets/Naver Sentiment Movie Corpus/ratings_train.txt")
    ratings_tr = ratings_tr.dropna(subset=["document"])

    sents = ratings_tr["document"].tolist()
    sents = [re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ", i) for i in sents]

    okt = Okt()
    corpus = [okt.morphs(i, stem=True, norm=True) for i in tqdm(sents)]
    with open(path, mode="w", encoding="utf-8") as f:
        json.dump(corpus, fp=f, ensure_ascii=False, indent="\t")

In [33]:
filename = "ratings_train_okt_w2v"
if os.path.exists(filename):
    model = gensim.models.KeyedVectors.load_word2vec_format(filename)
else:
    model = gensim.models.Word2Vec(corpus, vector_size=100, window=5, min_count=5, workers=4, sg=0)
    model.wv.save_word2vec_format(filename)

In [36]:
model.wv.most_similar("미국")

[('중국', 0.8507562875747681),
 ('일본', 0.8147351145744324),
 ('조폭', 0.7481272220611572),
 ('영웅', 0.743746817111969),
 ('영국', 0.7401779294013977),
 ('한국', 0.7373713850975037),
 ('문화', 0.732636034488678),
 ('헐리우드', 0.7285153269767761),
 ('일본인', 0.7165097594261169),
 ('유럽', 0.7122341990470886)]

# Visualization

In [39]:
!python -m gensim.scripts.word2vec2tensor --input ratings_train_okt_w2v --output ratings_train_okt_w2v

C:\Users\82104\anaconda3\envs\tf2.3\python.exe: Error while finding module specification for 'gensim.scripts.word2vec2tensor' (ModuleNotFoundError: No module named 'gensim')


- Word2Vec 모델 `rating_train_w2v`이 존재할 때 위 코드를 실행하면 `ratings_train_w2v_tensor.tsv`, `ratings_train_w2v_metadata.tsv`이 생성됩니다.
- 다음 페이지에 접속합니다 : https://projector.tensorflow.org/
- 위에 있는 Choose file 버튼을 누르고 ted_en_w2v_tensor.tsv 파일을 업로드하고, 아래에 있는 Choose file 버튼을 누르고 ted_en_w2v_metadata.tsv 파일을 업로드합니다. 두 파일을 업로드하면 임베딩 프로젝터에 학습했던 워드 임베딩 모델이 시각화됩니다.